In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [4]:
'''from google.colab import files
path_to_file = list(files.upload().keys())[0]'''
#To open your own file in Google Colab

'from google.colab import files\npath_to_file = list(files.upload().keys())[0]'

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
print(text[:250]) #First 250 characters of file(play)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Encoding

In [6]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)} 
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [10]:
print('{')
for char,_ in zip(char2idx,range(5)):
  print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  ...



In [11]:
print('{} -- char mapping to int -> {}'.format(repr(text[:13]), text_to_int(text[:13])))

'First Citizen' -- char mapping to int -> [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [12]:
#Integer to Text
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:25]))

First Citizen:
Before we 


Training Examples (because it's not feasible to pass all characters(approx 1 mil) of play to model)

In [13]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(10):
    print(idx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i


In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) #batch data into length of 101 & drop rest
for i in sequences.take(13):
    print(repr(''.join(idx2char[i.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
'zens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but th'
'e superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we a'
're too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particula'
'rise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we bec'
'ome rakes: for the gods kn

In [15]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [16]:
BATCH_SIZE = 64
VOCAB_SIZE=len(vocab)
EMBEDDING_DIM=256
RNN_UNITS=1024
BUFFER_SIZE=10000
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int32, tf.int32)>

Model

In [21]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_3 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [25]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(f"(batch_size, sequence_length, vocab_size {example_batch_predictions.shape}")  # print out the output shape

(batch_size, sequence_length, vocab_size (64, 100, 65)


In [23]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
#print(len(example_batch_predictions))
#print(example_batch_predictions)

In [26]:
# lets examine one prediction
pred=example_batch_predictions[0]
#print(len(pred))
#print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

In [27]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
#print(len(time_pred))
#print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

In [28]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

#sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
#sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
#predicted_chars = int_to_text(sampled_indices)

#predicted_chars  # and this is what the model predicted for training sequence 1

In [29]:
#def loss(labels, logits):
#  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [30]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1743236, shape=(), dtype=float32)


In [31]:
tf.exp(example_batch_mean_loss).numpy()

64.99586

In [32]:
model.compile(optimizer='adam',loss=loss)

In [33]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [34]:
history = model.fit(data, epochs=3, callbacks=[checkpoint_callback])

Epoch 1/3
172/172 [==============================] - 412s 2s/step - loss: 2.6514
Epoch 2/3
172/172 [==============================] - 420s 2s/step - loss: 1.9577
Epoch 3/3
172/172 [==============================] - 433s 3s/step - loss: 1.6902


In [35]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_3'

In [36]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [37]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


Text Generation

In [44]:
def generate_text(model, start_string):
  num_generate = 800
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 1.0
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [45]:
inp = input("Type to generate a random piece of play: ")
print(generate_text(model, inp))

hellore?
With wibless was the wird; in thy liening of he,
Parrokell, ond ene you, slive throw mine unstraget.

ADLAND:
'Tis thy good hath mares willingle.

LUCIO:
If that gose! why on these dearm quelieve ese, cilith
His vainest us more'st being on mine
Herours bain evers most sot.

PETRUCHIO:
Garing, forlow his head; My lord,
And they will with slizes me with unn off ROCENES:
Well, not his gentlemandy.

LEONTE:
O Guezen make be agroo, whoF paint teepled.

CAMILLO:
Yet as revirem these beying man,
And may rade hank our grace:
They soul shall deservice this kilds our fairet,
But to be his brabutey shall me will flie.

MISAMIA:
Who, this, those to know,
The voll to leave tide night tonguet
Low mar, hath han marrer'd to reveel
To themble beld the faken this rigist, Shower!

GLOUCESTER:
And lord ag
